In [11]:
#Imports NumPy used for mathematical operations, copy module for making copies
import copy, numpy as np

#Creating a random seed that will always have random numbers
np.random.seed()

In [12]:
#Input variables for the gradient, inputs, outputs and hidden layers
alpha   = 1
inputs  = 2    #How many inputs there are
hidden  = 16   #How many inputs to the hidden layer
outputs = 1    #Required number of outputs

In [13]:
#Generating the dataset to be used within training
#Integer to binary and binary length
int_bin = {}
bin_len = 8

#Determining the largest number
l_num   = pow(2, bin_len)
bin_num = np.unpackbits(np.array([range(l_num)], dtype = np.uint8).T, axis = 1)

#Converting largest number to binary
for i in range(l_num):
    int_bin[i] = bin_num[i]

In [14]:
#Sigmoid nonlinearity Function, returns a float
def sig(x):
    sig_num = 1 / (1 + np.exp(-x))
    return sig_num

#Converts the sigmoid result into its derivative
def deriv(sig_num):
    return sig_num * (1 - sig_num)

In [15]:
#Weight initialization for the network
#Synapse weights are randomised base on what layer they are connected to
syn_0 = 2 * np.random.random((inputs, hidden))  - 1
syn_1 = 2 * np.random.random((hidden, outputs)) - 1
syn_h = 2 * np.random.random((hidden, hidden))  - 1

#Updating the weights
update_0 = np.zeros_like(syn_0)
update_1 = np.zeros_like(syn_1)
update_h = np.zeros_like(syn_h)

#Training the neural network
for j in range(10000):
    
    #Creating addition to train on
    int_1 = np.random.randint(l_num/2) #Random integer
    bin_1 = int_bin[int_1] #Converting the integer to binary
    
    int_2 = np.random.randint(l_num/2)
    bin_2 = int_bin[int_2]
    
    int_3 = int_1 + int_2 #Integer answer
    bin_3 = int_bin[int_3] #Binary answer
    
    #"Best guess"
    guess = np.zeros_like(bin_3)
    
    #Storing the error
    error = 0
    
    #Layer 1 values and layer 2 delta
    lay2_del = list()
    lay1_val = list()
    lay1_val.append(np.zeros(hidden))
    
    #Iteration through the encoded binary - Forward propagation
    for pos in range(bin_len):
        
        #Input into the network
        X = np.array([[bin_1[bin_len - pos - 1], bin_2[bin_len - pos - 1]]])
        #Output of the network
        Y = np.array([[bin_3[bin_len - pos - 1]]]).T
        
        #The hidden layer
        lay_1 = sig(np.dot(X, syn_0) + np.dot(lay1_val[-1], syn_h))
        
        #Output layer
        lay_2 = sig(np.dot(lay_1, syn_1))
        
        #Error checking the result
        lay2_error = Y - lay_2
        lay2_del.append((lay2_error) * deriv(lay_2))
        error += np.abs(lay2_error[0])
        
        #Decode the answer into decimal
        guess[bin_len - pos - 1] = np.round(lay_2[0][0])
        
        #Creating a deep copy of the hidden layer to use it next
        lay1_val.append(copy.deepcopy(lay_1))
    
    #Creating a new delta for hidden layer based on the inital variable
    new_delta = np.zeros(hidden)
    
    #Shifting the binary and error checking each layer - Backpropagation
    for pos in range(bin_len):
        
        X      = np.array([[bin_1[pos], bin_2[pos]]])
        lay_1  = lay1_val[-pos - 1]
        old_l1 = lay1_val[-pos - 2]
        
        #Output error in the shift
        lay2_delta = lay2_del[-pos - 1]
        
        #Hidden layer error
        lay1_delta = (new_delta.dot(syn_h.T) + lay2_delta.dot(syn_1.T)) * deriv(lay_1)
        
        #Updating the weights - Output, Hidden and then Input connections
        update_1 += np.atleast_2d(lay_1).T.dot(lay2_delta)
        update_h += np.atleast_2d(old_l1).T.dot(lay1_delta)
        update_0 += X.T.dot(lay1_delta)
        
        #Update the new delta for the hidden layer
        new_delta = lay1_delta

    #New synapse weights after each cycle - multipled by the alpha
    syn_0 += update_0 * alpha
    syn_h += update_h * alpha
    syn_1 += update_1 * alpha
    
    #Reset the updates to zero
    update_0 *= 0
    update_h *= 0
    update_1 *= 0
    
    #Print the progress of the network
    if(j% 1000 == 0):
        print ("\nThe error: " + str(error) + "\nGuess: " + str(guess) + "\nCorrect answer: " + str(bin_3))
        
        #Reversing the number and printing out the answer
        answer = 0
        for index, x in enumerate(reversed(guess)):
            answer += x * pow(2, index)
        
        print (str(int_1) + " + " + str(int_2) + " = " + str(answer))


The error: [ 5.8158042]
Guess: [1 1 1 1 1 1 1 1]
Correct answer: [0 0 0 1 0 1 0 0]
20 + 0 = 255

The error: [ 3.35752997]
Guess: [0 1 0 0 0 0 0 0]
Correct answer: [0 1 1 0 0 0 1 0]
59 + 39 = 64

The error: [ 0.330374]
Guess: [0 1 0 0 0 0 1 1]
Correct answer: [0 1 0 0 0 0 1 1]
40 + 27 = 67

The error: [ 0.17629542]
Guess: [1 1 0 0 1 1 0 1]
Correct answer: [1 1 0 0 1 1 0 1]
98 + 107 = 205

The error: [ 0.10893389]
Guess: [0 0 0 1 1 0 1 0]
Correct answer: [0 0 0 1 1 0 1 0]
8 + 18 = 26

The error: [ 0.16986997]
Guess: [1 1 1 1 0 1 0 0]
Correct answer: [1 1 1 1 0 1 0 0]
117 + 127 = 244

The error: [ 0.07941621]
Guess: [0 1 0 1 1 1 0 1]
Correct answer: [0 1 0 1 1 1 0 1]
64 + 29 = 93

The error: [ 0.08722026]
Guess: [0 0 1 1 1 0 1 0]
Correct answer: [0 0 1 1 1 0 1 0]
40 + 18 = 58

The error: [ 0.08757574]
Guess: [1 0 0 0 1 0 0 0]
Correct answer: [1 0 0 0 1 0 0 0]
11 + 125 = 136

The error: [ 0.07609397]
Guess: [0 1 0 0 0 0 0 0]
Correct answer: [0 1 0 0 0 0 0 0]
12 + 52 = 64
